Import Dependencies

In [2]:
import urllib
from bs4 import BeautifulSoup as bs
import nltk
import pandas  as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from nltk import ngrams

In [3]:
df = pd.read_csv('news_summary.csv')
i = 3120

Extract data from html files

In [4]:
url = df['read_more'][i]
gold_summary = df['text'][i]
if "indiatoday" in url or "intoday" in url:
    baseurl = "i"
elif "hindustantimes" in url:
    baseurl = "h"
elif "theguardian" in url:
    baseurl = "t"
file = urllib.urlopen(url)
html = file.read()

<h2>Parse the data to extract article text</h2>

In [5]:
soup = bs(html, 'html.parser')

if baseurl == "i":
    entries = soup.find_all('div',{'class':'description'})
elif baseurl == "t":
    entries = soup.find_all('div',{'itemprop':'articleBody'})
elif baseurl == "h":
    entries = soup.find_all('div',{'itemprop':'articlebody'})

for each in entries:
        if each.figure:
            each.figure.decompose()
content = []
for e in entries:
     content.extend(e.find_all("p"))

text = ""
for each in content:
    text = text + each.get_text() +" "
    
text = text.encode('utf-8').decode("unicode_escape").encode('ascii','ignore')
print text
text = nltk.sent_tokenize(text)    
    
# for each in text:
#     print each , "\n"

Start extraction

In [471]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter

In [472]:
# tokenize the article
tokenizer = RegexpTokenizer(r'\w+')
word_list=[]
for t in text:
    word_list.extend(tokenizer.tokenize(t))


# remove stop words from the article
filtered_words = [word for word in word_list if word.lower() not in stopwords.words('english')]
total_words = len(filtered_words)
words = filtered_words

# find n-gram probability for filtered words
filtered_words  = []
for each in Counter(words).items():
    filtered_words.append([each[0] , float(each[1])/float(total_words)])

# identify important words
important_words = []
for item in filtered_words:
    if item[1]>0.003:
        important_words.append(item[0])

while '. ' in text:
    text = text.replace('. ','\n') 
# calculate sentence score
while "\n" in text:
    text = text.split("\n")

In [473]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [474]:
vect = TfidfVectorizer(stop_words='english')
dtm = vect.fit_transform(text)
features = vect.get_feature_names()

In [475]:
scores = zip(vect.get_feature_names(),
                 np.asarray(dtm.sum(axis=0)).ravel())
sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
# for item in sorted_scores:
#     print "{0:50} Score: {1}".format(item[0], item[1])

In [476]:
# import numpy as np

# #review_id = np.random.randint(0, len(text))
# review_text = words
# review_length = len(words)

# # create a dictionary of words and their TF-IDF scores
# word_scores = {}
# for word in words:
# #     word = word.lower()
#     if word in features:
# #         print word
# #         print dtm[review_id, features.index(word)]
#          #word_scores[word] = dtm[review_id, features.index(word)]
#          word_scores[word] = dtm[10, features.index(word)]

# print word_scores
# # print words with the top 5 TF-IDF scores
# print 'TOP SCORING WORDS:'
# top_scores = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[:10]
# for word, score in top_scores:
#     print word

In [477]:
sorted_scores = dict(sorted_scores)


In [478]:
sentence_score = []
for each in text:
    score = 0.0
#     each = each.
    for word in each.split(" "):
        #print word
        word = word.lower()
        if word in sorted_scores:
#             print "Hello"
            score += sorted_scores[word]
#             print word_scores[word]
    sentence_score.append([each,score])    
        

In [479]:
machine_summary = ""
sentences = []
for each in sentence_score:
    if each[1] != 0.0:
        machine_summary = machine_summary + each[0]
        sentences.append([each[0],each[1]])
        

In [480]:
def rougeN(gold_summary,machine_summary,n):
    stemmer = SnowballStemmer("english")

    gold_summary_slammed = ""
    for word in gold_summary.split():
        gold_summary_slammed = gold_summary_slammed+stemmer.stem(word)+" "


    machine_summary_slammed = ""
    for word in machine_summary.split():
        machine_summary_slammed = machine_summary_slammed+stemmer.stem(word)+" "

    n_gold = ngrams(gold_summary_slammed.split(" "),n)
    n_machine = ngrams(machine_summary_slammed.split(" "),n)

    gold_list = []

    for gram in n_gold:
        gold_list.append(gram)

    machine_list = []
    for gram in n_machine:
        machine_list.append(gram)

    return float(len(list(set(gold_list).intersection(machine_list))))/float(len(list(set(gold_list))))

In [481]:
scores = []
for SUMMARY_COMP_FACT in range(1,len(sentences)):
    top_sentences = sorted(dict(sentences).items(), key=lambda x: x[1], reverse=True)[:(len(sentences)/SUMMARY_COMP_FACT)]
    machine_summary = ""
    for each in top_sentences:
#         print each
        machine_summary = machine_summary + each[0]
    scores.append([SUMMARY_COMP_FACT, len(top_sentences), rougeN(gold_summary,machine_summary,2), rougeN(gold_summary,machine_summary,1)])
        #print top_sentences[each],each

In [482]:
# scores1 = np.array(scores)
# print scores1
# # val = np.average(scores1[:,2])
# for i in range(0,len(scores1[:])-1):
#     diff = scores1[i,2]-scores1[i+1,2]
#     print diff, scores1[i,1]
#     if (diff > 0.15):
#         num_sen =  scores1[i,1]
#         break
    

In [483]:
# summary = sorted(dict(sentences).items(), key=lambda x: x[1], reverse=True)[:5]
# for s in summary:
#     print s[0]

In [491]:
# arr = np.array(sentences)
# arr
paired_sens = {}
num_sen = len(sentences)/8

for pair in enumerate(sentences):
     paired_sens[pair[0]]=pair[1][1]
        
temp = sorted((paired_sens).items(), key=lambda x: x[1], reverse=True)[:int(num_sen)]
temp = sorted(dict(temp).items(), key=lambda x: x[0], reverse=False)[:int(num_sen)]
machine_summary = ""
for i in temp:
    print sentences[i[0]][0],"\n"
    machine_summary = machine_summary +  sentences[i[0]][0] + " "
print rougeN(gold_summary,machine_summary,2),rougeN(gold_summary,machine_summary,1)

Salman Khan will never move out of his flat into a bigger bungalow because he has many memories attached to his home. 

At 51, Salman is now tackling yet another 'panga': Tiger Zinda Hai and Remo D'Souza's dance film. 

Salman said, "After Sultan, there was lot of pain in my body... Now I have signed Tiger Zinda Hai like a fool, I am jumping off the building, running and shooting, doing all action; I am going mad in the film. 

Contrary to reports, Salman Khan has not signed ABCD 3 opposite Jacqueline Fernandez. 

Even though Salman Khan and his Tubelight director Kabir Khan share a great rapport, Salman says he will not accept any and every film that Kabir gives him. 

5 intriguing points from this Salman film ALSO WATCH: A look at Salman Khan's controversial life 

0.314814814815 0.571428571429


In [485]:
# def find_nearest(array,value):
#     idx = (np.abs(array-value)).argmin()
#     return idx,array[idx]

In [486]:
# SUMMARY_COMP_FACT = find_nearest(scores1[:,2],val)[0]

In [487]:
# top_sentences = sorted(dict(sentences).items(), key=lambda x: x[1], reverse=True)[:(len(sentences)/SUMMARY_COMP_FACT)]
# machine_summary = ""
# for each in top_sentences:
#     print each
#     machine_summary = machine_summary + each[0]
# print SUMMARY_COMP_FACT, len(top_sentences), rougeN(gold_summary,machine_summary,2), rougeN(gold_summary,machine_summary,1)

In [488]:
# machine_summary = "An alleged suspect in a kidnapping case was found hanging inside the washroom of the Jahangirpuri police station in north Delhi on Wednesday, hours after he was called by the cops for interrogation. Their relationship ended a few months ago and Kumar quit the job and returned to his village, said a police officer. The woman went missing a few months ago, prompting her husband to file a police complaint and a habeas corpus petition at the Delhi High Court. A team was sent to Kumars village but when he was found unavailable, he was instructed to report to the Jahangirpuri police station. He willingly arrived in Delhi on Tuesday and visited the police station, said Kumars friend, Dharmendra, who dropped him at the police station around 3 pm."
# print rougeN(gold_summary,machine_summary,2)